In [1]:
import pandas as pd
import time as time
import requests

In [2]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [3]:
# This function scrapes at least min_posts from Reddit not including [removed] 
# and [deleted] posts, and contingent on whether a post is_self or not

def scrape_reddit(subreddit, min_posts, is_self):
     
    url = 'https://api.pushshift.io/reddit/search/submission'
    df = pd.DataFrame()

    while len(df) < min_posts:
        try:
            if is_self == 'Y':
                if len(df) == 0:
                    params = {
                        'subreddit': subreddit,
                        'size': 100,
                        'before': int(time.time()),
                        'is_self': True,
                    }
                    res = requests.get(url, params)
                    data = res.json()
                    posts = data['data']
                    df = pd.DataFrame(posts)
                    df.drop(df[df['selftext'] == '[removed]'].index, inplace = True)
                    df.drop(df[df['selftext'] == '[deleted]'].index, inplace = True)
                else:
                    params = {
                        'subreddit': subreddit,
                        'size': 100,
                        'before': df.iloc[-1]['created_utc'],
                        'is_self': True,
                    }
                    res = requests.get(url, params)
                    data = res.json()
                    posts = data['data']
                    df2 = pd.DataFrame(posts)
                    df = df.append(df2, ignore_index=True)
                    df.drop(df[df['selftext'] == '[removed]'].index, inplace = True)
                    df.drop(df[df['selftext'] == '[deleted]'].index, inplace = True)        
            elif is_self == 'N':
                if len(df) == 0:
                    params = {
                        'subreddit': subreddit,
                        'size': 100,
                        'before': int(time.time()),
                    }
                    res = requests.get(url, params)
                    data = res.json()
                    posts = data['data']
                    df = pd.DataFrame(posts)
                    df.drop(df[df['selftext'] == '[removed]'].index, inplace = True)
                    df.drop(df[df['selftext'] == '[deleted]'].index, inplace = True)
                else:
                    params = {
                        'subreddit': subreddit,
                        'size': 100,
                        'before': df.iloc[-1]['created_utc'],
                    }
                    res = requests.get(url, params)
                    data = res.json()
                    posts = data['data']
                    df2 = pd.DataFrame(posts)
                    df = df.append(df2, ignore_index=True)
                    df.drop(df[df['selftext'] == '[removed]'].index, inplace = True)
                    df.drop(df[df['selftext'] == '[deleted]'].index, inplace = True)
            else:
                print(f"Enter 'Y' or 'N' for third argument.")
                break
        except JSONDecodeError:
            pass
        except:
            print('Other error.')
            
        time.sleep(3)
        
    df.reset_index(inplace=True)
    return df

In [4]:
# CHANGE THIS CELL FOR PULLING NEW INFORMATION

subreddit = 'askwomen'

In [5]:
df = scrape_reddit(subreddit, 10_000, 'Y')

In [6]:
df[['title', 'selftext', 'author', 'subreddit']].to_csv(f'./{subreddit}.csv', index=False)